# Methods and dispatch



Suppose that we define a function to find the midpoint of an interval whose endpoints are given.

	


In [1]:
function midpoint(a,b)
	(a+b)/2
end

midpoint(-1,2)

0.5

We might want to find the 'midpoint' of other things as well, such as a rectangle given in terms of its opposing corners.


In [2]:
function midpoint(x₁,y₁,x₂,y₂)
	midpoint(x₁,x₂),midpoint(y₁,y₂)
end

midpoint(-1,-1,1,2)

(0.0, 0.5)


How was this possible? While we have just one function `midpoint`, we have defined two *methods* for it. They can be distinguished by their argument signatures.


In [3]:
methods(midpoint)

# 2 methods for generic function "midpoint":
[1] midpoint(a, b) in Main at In[1]:1
[2] midpoint(x₁, y₁, x₂, y₂) in Main at In[2]:1


Suppose we also want to accept an interval input as a tuple. If we are not worried about invalid inputs, we can define a new method for one input argument and reuse our previous definition by applying the splat operation.


In [5]:
midpoint(interval) = midpoint(interval...)
methods(midpoint)

# 3 methods for generic function "midpoint":
[1] midpoint(interval) in Main at In[5]:1
[2] midpoint(a, b) in Main at In[1]:1
[3] midpoint(x₁, y₁, x₂, y₂) in Main at In[2]:1

In [6]:
midpoint( (-1,2) )

0.5

If we are willing to specify each corner of a rectangle using a vector for its coordinates, we can continue to use the existing method for two input arguments, since `+` has built-in methods for both numbers and vectors.


In [7]:
midpoint([-1,-1],[1,2])

2-element Vector{Float64}:
 0.0
 0.5

## Argument types 

Now suppose we want to extend the `midpoint` function to work for a circle. The natural way to specify a circle is by a vector of coordinates to represent its center, and a number to represent its radius.

We already have a method definition for two input arguments. However, we can be more specific about the *types* of those arguments in order to distinguish the new case.


In [8]:
midpoint(center::Vector{Float64},radius::Float64) = center
methods(midpoint)

# 4 methods for generic function "midpoint":
[1] midpoint(center::Vector{Float64}, radius::Float64) in Main at In[8]:1
[2] midpoint(interval) in Main at In[5]:1
[3] midpoint(a, b) in Main at In[1]:1
[4] midpoint(x₁, y₁, x₂, y₂) in Main at In[2]:1

In [9]:
midpoint([-3.5,2],2.5)

2-element Vector{Float64}:
 -3.5
  2.0

The rule is that the most specific method definition that applies is used first. This process is called *dispatch*, and Julia's notion of multiple dispatch is unusual and powerful. It makes types a central focus of the language.

## Parametric types

We still have a problem in our simple example: not all numeric values are of type `Float64`.

In [10]:
midpoint([-3,2],2.5)

LoadError: MethodError: no method matching +(::Vector{Int64}, ::Float64)
For element-wise addition, use broadcasting with dot syntax: array .+ scalar
[0mClosest candidates are:
[0m  +(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m) at operators.jl:591
[0m  +([91m::T[39m, ::T) where T<:Union{Float16, Float32, Float64} at float.jl:383
[0m  +([91m::Base.TwicePrecision[39m, ::Number) at twiceprecision.jl:290
[0m  ...

One option is have another method that forces a conversion of the vector to `Float64`. But that solution is not friendly to other floating-point types, and conceptually it should not be necessary. Or, we could write a new definition for `Int64`, of course, but then we have other possible precisions, and these can all be combined with different numeric types for the radius as well. 

To avoid these issues, we just want to specify that the types "act like real numbers," and this is possible.

In [17]:
function midpoint(center::Vector{<:Real},radius::Real)
    return center
end

midpoint([-3,2],2.5)

2-element Vector{Int64}:
 -3
  2

We have used a hierarchy of types via the `<:` relation. These can be explored explicitly.

In [18]:
subtypes(Real)

4-element Vector{Any}:
 AbstractFloat
 AbstractIrrational
 Integer
 Rational

An `AbstractFloat` is any type that "quacks like" a floating-point value. Yes, this is called *duck typing*, because there is no formal definition.

In [19]:
subtypes(AbstractFloat)

4-element Vector{Any}:
 BigFloat
 Float16
 Float32
 Float64

In general, you want to specify as abstract or high-level a type as is reasonable. For example, there are other implementations of "vectors" that useful in certain circumstances, such as memory management of fixed-length vectors. A better definition of our method would be:

In [20]:
function midpoint(center::AbstractVector{<:Real},radius::Real)
    return center
end

midpoint (generic function with 6 methods)

We might also want to allow the center to be specified, and returned, as a complex number.

In [27]:
midpoint(center::Complex,radius::Real) = center

midpoint (generic function with 7 methods)

In [28]:
midpoint(1+2im,1.0)

1 + 2im

In [29]:
methods(+)

# 206 methods for generic function "+":
[1] +(x::T, y::T) where T<:Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8} in Base at int.jl:87
[2] +(x::T, y::T) where T<:Union{Float16, Float32, Float64} in Base at float.jl:383
[3] +(c::Union{UInt16, UInt32, UInt64, UInt8}, x::BigInt) in Base.GMP at gmp.jl:531
[4] +(c::Union{Int16, Int32, Int64, Int8}, x::BigInt) in Base.GMP at gmp.jl:537
[5] +(c::Union{UInt16, UInt32, UInt64, UInt8}, x::BigFloat) in Base.MPFR at mpfr.jl:398
[6] +(c::Union{Int16, Int32, Int64, Int8}, x::BigFloat) in Base.MPFR at mpfr.jl:406
[7] +(c::Union{Float16, Float32, Float64}, x::BigFloat) in Base.MPFR at mpfr.jl:414
[8] +(x::Union{Dates.CompoundPeriod, Dates.Period}) in Dates at /Applications/Julia-1.8.app/Contents/Resources/julia/share/julia/stdlib/v1.8/Dates/src/periods.jl:378
[9] +(A::BitArray, B::BitArray) in Base at bitarray.jl:1180
[10] +(x::Dates.Instant) in Dates at /Applications/Julia-1.8.app/Contents/Resources/julia/share/julia/stdlib/v1.8/Dates/src/arithmetic.jl:4
[11] +(y::AbstractFloat, x::Bool) in Base at bool.jl:172
[12] +(A::LinearAlgebra.LowerTriangular, B::LinearAlgebra.LowerTriangular) in LinearAlgebra at /Applications/Julia-1.8.app/Contents/Resources/julia/share/julia/stdlib/v1.8/LinearAlgebra/src/triangular.jl:639
[13] +(A::LinearAlgebra.LowerTriangular, B::LinearAlgebra.UnitLowerTriangular) in LinearAlgebra at /Applications/Julia-1.8.app/Contents/Resources/julia/share/julia/stdlib/v1.8/LinearAlgebra/src/triangular.jl:641
[14] +(A::LinearAlgebra.LowerTriangular, B::LinearAlgebra.Bidiagonal) in LinearAlgebra at /Applications/Julia-1.8.app/Contents/Resources/julia/share/julia/stdlib/v1.8/LinearAlgebra/src/special.jl:98
[15] +(r::AbstractRange{<:Dates.TimeType}, x::Dates.Period) in Dates at /Applications/Julia-1.8.app/Contents/Resources/julia/share/julia/stdlib/v1.8/Dates/src/ranges.jl:64
[16] +(x::AbstractArray{<:Dates.TimeType}, y::Union{Dates.CompoundPeriod, Dates.Period}) in Dates at /Applications/Julia-1.8.app/Contents/Resources/julia/share/julia/stdlib/v1.8/Dates/src/deprecated.jl:6
[17] +(x::P, y::P) where P<:Dates.Period in Dates at /Applications/Julia-1.8.app/Contents/Resources/julia/share/julia/stdlib/v1.8/Dates/src/periods.jl:77
[18] +(x::Dates.Period, y::Dates.Period) in Dates at /Applications/Julia-1.8.app/Contents/Resources/julia/share/julia/stdlib/v1.8/Dates/src/periods.jl:367
[19] +(y::Dates.Period, x::Dates.CompoundPeriod) in Dates at /Applications/Julia-1.8.app/Contents/Resources/julia/share/julia/stdlib/v1.8/Dates/src/periods.jl:369
[20] +(y::Dates.Period, x::Dates.TimeType) in Dates at /Applications/Julia-1.8.app/Contents/Resources/julia/share/julia/stdlib/v1.8/Dates/src/arithmetic.jl:85
[21] +(x::Dates.Period, r::AbstractRange{<:Dates.TimeType}) in Dates at /Applications/Julia-1.8.app/Contents/Resources/julia/share/julia/stdlib/v1.8/Dates/src/ranges.jl:63
[22] +(y::Union{Dates.CompoundPeriod, Dates.Period}, x::AbstractArray{<:Dates.TimeType}) in Dates at /Applications/Julia-1.8.app/Contents/Resources/julia/share/julia/stdlib/v1.8/Dates/src/deprecated.jl:14
[23] +(A::LinearAlgebra.Symmetric, B::LinearAlgebra.Symmetric) in LinearAlgebra at /Applications/Julia-1.8.app/Contents/Resources/julia/share/julia/stdlib/v1.8/LinearAlgebra/src/symmetric.jl:469
[24] +(A::LinearAlgebra.Symmetric, B::LinearAlgebra.SymTridiagonal) in LinearAlgebra at /Applications/Julia-1.8.app/Contents/Resources/julia/share/julia/stdlib/v1.8/LinearAlgebra/src/symmetric.jl:486
[25] +(S::LinearAlgebra.Symmetric, D::LinearAlgebra.Diagonal) in LinearAlgebra at /Applications/Julia-1.8.app/Contents/Resources/julia/share/julia/stdlib/v1.8/LinearAlgebra/src/diagonal.jl:191
[26] +(A::LinearAlgebra.Symmetric{<:Any, <:SparseArrays.AbstractSparseMatrix}, B::SparseArrays.AbstractSparseMatrix) in SparseArrays at /Applications/Julia-1.8.app/Contents/Resources/julia/share/julia/stdlib/v1.8/SparseArrays/src/linalg.jl:15
[27] +(A::LinearAlgebra.Symmetric, B::SparseArrays.AbstractSparseMatri

In [32]:
using LinearAlgebra
methodswith(SymTridiagonal)

[1] /(A::SymTridiagonal, B::Number) in LinearAlgebra at /Applications/Julia-1.8.app/Contents/Resources/julia/share/julia/stdlib/v1.8/LinearAlgebra/src/tridiag.jl:212
[2] /(S::SymTridiagonal, D::Diagonal) in LinearAlgebra at /Applications/Julia-1.8.app/Contents/Resources/julia/share/julia/stdlib/v1.8/LinearAlgebra/src/diagonal.jl:498
[3] Bidiagonal(A::SymTridiagonal) in LinearAlgebra at /Applications/Julia-1.8.app/Contents/Resources/julia/share/julia/stdlib/v1.8/LinearAlgebra/src/special.jl:34
[4] Diagonal(A::SymTridiagonal) in LinearAlgebra at /Applications/Julia-1.8.app/Contents/Resources/julia/share/julia/stdlib/v1.8/LinearAlgebra/src/special.jl:30
[5] SymTridiagonal(S::SymTridiagonal) in LinearAlgebra at /Applications/Julia-1.8.app/Contents/Resources/julia/share/julia/stdlib/v1.8/LinearAlgebra/src/tridiag.jl:120
[6] Tridiagonal(A::SymTridiagonal) in LinearAlgebra at /Applications/Julia-1.8.app/Contents/Resources/julia/share/julia/stdlib/v1.8/LinearAlgebra/src/special.jl:37
[7] \(B::Number, A::SymTridiagonal) in LinearAlgebra at /Applications/Julia-1.8.app/Contents/Resources/julia/share/julia/stdlib/v1.8/LinearAlgebra/src/tridiag.jl:213
[8] \(D::Diagonal, S::SymTridiagonal) in LinearAlgebra at /Applications/Julia-1.8.app/Contents/Resources/julia/share/julia/stdlib/v1.8/LinearAlgebra/src/diagonal.jl:461
[9] \(T::SymTridiagonal, B::StridedVecOrMat) in LinearAlgebra at /Applications/Julia-1.8.app/Contents/Resources/julia/share/julia/stdlib/v1.8/LinearAlgebra/src/tridiag.jl:277
[10] adjoint(S::SymTridiagonal) in LinearAlgebra at /Applications/Julia-1.8.app/Contents/Resources/julia/share/julia/stdlib/v1.8/LinearAlgebra/src/tridiag.jl:164
[11] cholesky(S::SymTridiagonal) in LinearAlgebra at /Applications/Julia-1.8.app/Contents/Resources/julia/share/julia/stdlib/v1.8/LinearAlgebra/src/tridiag.jl:859
[12] cholesky(S::SymTridiagonal, ::NoPivot; check) in LinearAlgebra at /Applications/Julia-1.8.app/Contents/Resources/julia/share/julia/stdlib/v1.8/LinearAlgebra/src/tridiag.jl:859
[13] copyto!(dest::SymTridiagonal, src::SymTridiagonal) in LinearAlgebra at /Applications/Julia-1.8.app/Contents/Resources/julia/share/julia/stdlib/v1.8/LinearAlgebra/src/tridiag.jl:154
[14] copyto!(dest::SymTridiagonal, bc::Base.Broadcast.Broadcasted{<:LinearAlgebra.StructuredMatrixStyle}) in LinearAlgebra at /Applications/Julia-1.8.app/Contents/Resources/julia/share/julia/stdlib/v1.8/LinearAlgebra/src/structuredbroadcast.jl:183
[15] det(A::SymTridiagonal; shift) in LinearAlgebra at /Applications/Julia-1.8.app/Contents/Resources/julia/share/julia/stdlib/v1.8/LinearAlgebra/src/tridiag.jl:445
[16] diag(M::SymTridiagonal) in LinearAlgebra at /Applications/Julia-1.8.app/Contents/Resources/julia/share/julia/stdlib/v1.8/LinearAlgebra/src/tridiag.jl:190
[17] diag(M::SymTridiagonal, n::Integer) in LinearAlgebra at /Applications/Julia-1.8.app/Contents/Resources/julia/share/julia/stdlib/v1.8/LinearAlgebra/src/tridiag.jl:190
[18] dot(x::AbstractVector, S::SymTridiagonal, y::AbstractVector) in LinearAlgebra at /Applications/Julia-1.8.app/Contents/Resources/julia/share/julia/stdlib/v1.8/LinearAlgebra/src/tridiag.jl:256
[19] eigen(A::SymTridiagonal{T, V} where V<:AbstractVector{T}) where T in LinearAlgebra at /Applications/Julia-1.8.app/Contents/Resources/julia/share/julia/stdlib/v1.8/LinearAlgebra/src/tridiag.jl:284
[20] eigen(A::SymTridiagonal{T, V} where V<:AbstractVector{T}, irange::UnitRange) where T in LinearAlgebra at /Applications/Julia-1.8.app/Contents/Resources/julia/share/julia/stdlib/v1.8/LinearAlgebra/src/tridiag.jl:288
[21] eigen(A::SymTridiagonal{T, V} where V<:AbstractVector{T}, vl::Real, vu::Real) where T in LinearAlgebra at /Applications/Julia-1.8.app/Contents/Resources/julia/share/julia/stdlib/v1.8/LinearAlgebra/src/tridiag.jl:293
[22] eigmax(A::SymTridiagonal) in LinearAlgebra at /Applications/Julia-1.8.app/Contents/Resources/julia/share/julia/stdlib/v1.8/LinearAlgebra/src/tridiag.jl:310
[23] eigmin(A::SymTridiagonal) in LinearAlgebra at /Applications/